In [2]:
from enum import Enum

In [3]:
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

for member in Color:
    print(member.name, member.value, sep=": ")

RED: 1
GREEN: 2
BLUE: 3


In [4]:
from datetime import date

class Weekday(Enum):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 3
    THURSDAY = 4
    FRIDAY = 5
    SATURDAY = 6
    SUNDAY = 7

    @classmethod
    def today(cls):
        print(f"Today is {cls(date.today().isoweekday()).name}")    # use call syntax to return member by value

dir(Weekday.SATURDAY)

['__class__',
 '__doc__',
 '__eq__',
 '__hash__',
 '__module__',
 'name',
 'today',
 'value']

In [5]:
print(date.today().isoweekday())   # day 4th of the week (THURSDAY)
print(Weekday.today())

4
Today is THURSDAY
None


In [6]:
# Generate next value (staticmethod)
from enum import auto
from typing import Any 

class PowersOfThree(Enum):
    
    @staticmethod
    def _generate_next_value_(
        name: str,       # the name of the member being defined (called)
        start: int,      # the start value for the Enum; default is 1 
        count: int,      # the number of members currently defined, not including this one
        last_values: list[Any]    # a list of the previous values
    ) -> Any:
        """Determine the next value returned by auto.
        """
        return 3 ** (count + 1)
    
    FIRST = auto()
    SECOND = auto()

print(PowersOfThree.SECOND.value)
print(PowersOfThree.FIRST.value)

9
3


In [7]:
class Foo(Enum):
    """Trivical class used to experiment the _generate_next_value_.
    """

    @staticmethod
    def _generate_next_value_(
        name: str,     
        start: int,    
        count: int,
        last_values: list[Any],
    ) -> Any:
        return f"{name * start}-{name * count}-{last_values}"
    
    A = auto()
    B = auto()
    C = auto()

print(Foo.A.value)
print(Foo.B.value)
print(Foo.C.value)

A--[]
B-B-['A--[]']
C-CC-['A--[]', "B-B-['A--[]']"]


In [8]:
# __init__: does nothing, does not override old values

Weekday.__init__(*list(range(11, 18)))
for day in Weekday:
    print(day.name, day.value, sep=": ")

MONDAY: 1
TUESDAY: 2
WEDNESDAY: 3
THURSDAY: 4
FRIDAY: 5
SATURDAY: 6
SUNDAY: 7


In [32]:
# _missing_: a classmethod for looking up values not found in cls. 
# Default: does nothing. But can be overridden to implement custom search behavior.

from enum import StrEnum 

class Build(StrEnum):
    DEBUG = auto()
    OPTIMIZED = auto()

    @classmethod
    def _missing_(cls, value: str):   
        """This method is invoked when a value not found after perform normal lookup.
        """
        print("Run into _missing_ method.")
        value = value.lower()
        for member in cls:
            if member.value == value:
                return member
            
        return None    # cannot return value other than None 

print(Build.DEBUG.value)
print(Build("debug"))   # search member by value: does not run into _missing_ method (__call__ method)
print(Build("deBUG"))   # search member by value: run into _missing_ method
# print(Build("NotExist"))    # error instead of None 

debug
debug
Run into _missing_ method.
debug


In [33]:
Build.__call__("debug")

<Build.DEBUG: 'debug'>

In [37]:
from inspect import getsource

print(getsource(Enum.__call__)) 
print(getsource(Enum.__new__))

    def __call__(cls, value, names=_not_given, *values, module=None, qualname=None, type=None, start=1, boundary=None):
        """
        Either returns an existing member, or creates a new enum class.

        This method is used both when an enum class is given a value to match
        to an enumeration member (i.e. Color(3)) and for the functional API
        (i.e. Color = Enum('Color', names='RED GREEN BLUE')).

        The value lookup branch is chosen if the enum is final.

        When used for the functional API:

        `value` will be the name of the new class.

        `names` should be either a string of white-space/comma delimited names
        (values will start at `start`), or an iterator/mapping of name, value pairs.

        `module` should be set to the module this class is being created in;
        if it is not set, an attempt to find that module will be made, but if
        it fails the class will not be picklable.

        `qualname` should be set to the actual 

In [39]:
getsource(Enum.__init__)

'    def __init__(self, *args, **kwds):\n        pass\n'

In [41]:
print(getsource(Enum.__iter__))

    def __iter__(cls):
        """
        Return members in definition order.
        """
        return (cls._member_map_[name] for name in cls._member_names_)

